Courier charge Analayis:

In [ ]:
#import libraries:
import pandas as pd
import numpy as np
import math

In [ ]:
#import excel file companyx(L.H.S side):
companyX_SKU =pd.read_excel('/content/drive/MyDrive/cointab/cointab/Company X - SKU Master.xlsx')
companyX_order_report=pd.read_excel('/content/drive/MyDrive/cointab/cointab/Company X - Order Report.xlsx')

In [ ]:
companyX_SKU.dtypes

SKU           object
Weight (g)     int64
dtype: object

In [ ]:
companyX_SKU.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [ ]:
companyX_order_report.head(2)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0


In [ ]:
companyX_order_report.dtypes

ExternOrderNo      int64
SKU               object
Order Qty        float64
dtype: object

In [ ]:
#merge the table companyX(L.H.S):
companyX_mergedtable=pd.merge(companyX_order_report,companyX_SKU,how='inner',indicator=False)
companyX_mergedtable.head(2)

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127


In [ ]:
#create a new column calculate total weight of order :
companyX_mergedtable['order_weight(kg)'] = ((companyX_mergedtable['Order Qty']*companyX_mergedtable['Weight (g)'])/1000)
companyX_mergedtable.head(2)

,ExternOrderNo,SKU,Order Qty,Weight (g),order_weight(kg)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127


In [ ]:
companyX_final=companyX_mergedtable.groupby(by='ExternOrderNo',as_index=False).agg({'order_weight(kg)':'sum'})

In [ ]:
companyX_final.rename(columns={'ExternOrderNo':'Order ID','order_weight(kg)':'Total weight as per X (KG)'},inplace=True)
companyX_final

,Order ID,Total weight as per X (KG)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [ ]:
#calculate weight slab for company:
def ceil(x):
  f=0.5 * math.ceil(float(x)/0.5)
  return f

companyX_final['weight slab as per X (KG)'] = companyX_final['Total weight as per X (KG)'].apply(ceil)



In [ ]:
#read the excel file for courier(R.H.S)
courier_invoice=pd.read_excel('/content/drive/MyDrive/cointab/cointab/Courier Company - Invoice.xlsx')
companyX_pincode=pd.read_excel('/content/drive/MyDrive/cointab/cointab/Company X - Pincode Zones.xlsx')


In [ ]:
courier_invoice.head(2)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab charged by Courier Company (KG)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,1.0


In [ ]:
courier_invoice['Weight slab charged by Courier Company (KG)']=courier_invoice['Charged Weight'].apply(ceil)

In [ ]:
companyX_pincode.shape

(124, 3)

In [ ]:
# join the table for final pincode for L.H.S:
join_table=pd.merge(companyX_final,courier_invoice,how='inner',left_on='Order ID',right_on='Order ID',indicator=False)
pincode_from_couriertable=join_table[['Order ID','Customer Pincode']]

pincode_from_couriertable.shape

(124, 2)

In [ ]:
#found some duplicates so remove from pincode table:
companyX_pincode.drop_duplicates(inplace=True)

In [ ]:
pincode_table=pd.merge(pincode_from_couriertable,companyX_pincode,how='left',left_on='Customer Pincode',right_on='Customer Pincode',indicator=False)
pincode_table


,Order ID,Customer Pincode,Warehouse Pincode,Zone
0,2001806210,140604,121003,b
1,2001806226,723146,121003,d
2,2001806229,421204,121003,d
3,2001806232,507101,121003,d
4,2001806233,263139,121003,b
...,...,...,...,...
119,2001821995,342008,121003,b
120,2001822466,342301,121003,b
121,2001823564,492001,121003,d
122,2001825261,517128,121003,d


In [ ]:
pincode_from_couriertable.shape

#companyX_pincode.shape


(124, 2)

In [ ]:
# final Company side Table:
company_xfinal=pd.merge(pincode_table,companyX_final,how='inner',left_on='Order ID',right_on='Order ID',indicator=False)
company_xfinal

,Order ID,Customer Pincode,Warehouse Pincode,Zone,Total weight as per X (KG),weight slab as per X (KG)
0,2001806210,140604,121003,b,0.220,0.5
1,2001806226,723146,121003,d,0.480,0.5
2,2001806229,421204,121003,d,0.500,0.5
3,2001806232,507101,121003,d,1.302,1.5
4,2001806233,263139,121003,b,0.245,0.5
...,...,...,...,...,...,...
119,2001821995,342008,121003,b,0.477,0.5
120,2001822466,342301,121003,b,1.376,1.5
121,2001823564,492001,121003,d,0.672,1.0
122,2001825261,517128,121003,d,1.557,2.0


In [ ]:
#rename some columns as per the results:
company_xfinal.rename(columns={'Zone':'Delivery Zone as per X'},inplace=True)

In [ ]:
#combined table for calculation:
final=pd.merge(company_xfinal,courier_invoice,how='inner',left_on='Order ID',right_on='Order ID',indicator=False)

In [ ]:
final.tail(5)

,Order ID,Customer Pincode_x,Warehouse Pincode_x,Delivery Zone as per X,Total weight as per X (KG),weight slab as per X (KG),AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab charged by Courier Company (KG)
119,2001821995,342008,121003,b,0.477,0.5,1091121183730,0.5,121003,342008,d,Forward charges,45.4,0.5
120,2001822466,342301,121003,b,1.376,1.5,1091121305541,1.1,121003,342301,d,Forward charges,135.0,1.5
121,2001823564,492001,121003,d,0.672,1.0,1091121666133,0.7,121003,492001,d,Forward and RTO charges,172.8,1.0
122,2001825261,517128,121003,d,1.557,2.0,1091121981575,1.6,121003,517128,d,Forward and RTO charges,345.0,2.0
123,2001827036,173213,121003,e,2.176,2.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0


In [ ]:
# read the rate file for charges:
rate=pd.read_excel('/content/drive/MyDrive/cointab/cointab/Courier Company - Rates.xlsx')
rate

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [ ]:
def expected_rate(x,y,z):
    if(x =='a'):
      if( y =='Forward charges'):
        t1=z/0.5
        fwd=rate.iloc[0,0]
        fwd_additional=(t1-1)*rate.iloc[0,1]
        total=fwd+fwd_additional
        return round(total,2)      
      else:
        t1=z/0.5
        fwd=rate.iloc[0,0]
        fwd_additional=(t1-1)*rate.iloc[0,1]
        rdo=1*rate.iloc[0,10]
        rdo_additional=(t1-1)*rate.iloc[0,11]
        total=fwd+fwd_additional+rdo+rdo_additional
        return round(total,2)
    if(x =='b'):
      if( y =='Forward charges'):
        t1=z/0.5
        fwd=rate.iloc[0,2]
        fwd_additional=(t1-1)*rate.iloc[0,3]
        total=fwd+fwd_additional
        return round(total,2)      
      else:
        t1=z/0.5
        fwd=rate.iloc[0,2]
        fwd_additional=(t1-1)*rate.iloc[0,3]
        rdo=1*rate.iloc[0,12]
        rdo_additional=(t1-1)*rate.iloc[0,13]
        total=fwd+fwd_additional+rdo+rdo_additional
        return round(total,2)  
    if(x =='c'):
      if( y =='Forward charges'):
        t1=z/0.5
        fwd=rate.iloc[0,4]
        fwd_additional=(t1-1)*rate.iloc[0,5]
        total=fwd+fwd_additional
        return round(total,2)      
      else:
        t1=z/0.5
        fwd=rate.iloc[0,4]
        fwd_additional=(t1-1)*rate.iloc[0,5]
        rdo=1*rate.iloc[0,14]
        rdo_additional=(t1-1)*rate.iloc[0,15]
        total=fwd+fwd_additional+rdo+rdo_additional
        return round(total,2)
    if(x =='d'):
      if( y =='Forward charges'):
        t1=z/0.5
        fwd=rate.iloc[0,6]
        fwd_additional=(t1-1)*rate.iloc[0,7]
        total=fwd+fwd_additional
        return round(total,2)      
      else:
        t1=z/0.5
        fwd=rate.iloc[0,6]
        fwd_additional=(t1-1)*rate.iloc[0,7]
        rdo=1*rate.iloc[0,16]
        rdo_additional=(t1-1)*rate.iloc[0,17]
        total=fwd+fwd_additional+rdo+rdo_additional
        return round(total,2)
    if(x =='e'):
      if( y =='Forward charges'):
        t1=z/0.5
        fwd=rate.iloc[0,8]
        fwd_additional=(t1-1)*rate.iloc[0,9]
        total=fwd+fwd_additional
        return round(total,2)
      else:
        t1=z/0.5
        fwd=rate.iloc[0,8]
        fwd_additional=(t1-1)*rate.iloc[0,9]
        rdo=1*rate.iloc[0,18]
        rdo_additional=(t1-1)*rate.iloc[0,19]
        total=fwd+fwd_additional+rdo+rdo_additional
        return round(total,2)
    
    
        
    
final['Expected_charge as per x(Rs)']=final.apply(lambda x: expected_rate(x['Delivery Zone as per X'],x['Type of Shipment'],x['weight slab as per X (KG)']),axis=1)

#expected_rate("e","Forward cha",5)

In [ ]:
final.head(10)

,Order ID,Customer Pincode_x,Warehouse Pincode_x,Delivery Zone as per X,Total weight as per X (KG),weight slab as per X (KG),AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Zone,Type of Shipment,Billing Amount (Rs.),Weight slab charged by Courier Company (KG),Expected_charge as per x(Rs)
0,2001806210,140604,121003,b,0.220,0.5,1091117221940,2.92,121003,140604,b,Forward charges,174.5,3.0,33.0
1,2001806226,723146,121003,d,0.480,0.5,1091117222065,0.68,121003,723146,d,Forward charges,90.2,1.0,45.4
2,2001806229,421204,121003,d,0.500,0.5,1091117222080,0.71,121003,421204,d,Forward charges,90.2,1.0,45.4
3,2001806232,507101,121003,d,1.302,1.5,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.5,135.0
4,2001806233,263139,121003,b,0.245,0.5,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0,33.0
5,2001806251,743263,121003,d,0.245,0.5,1091117222146,1.27,121003,743263,d,Forward charges,135.0,1.5,45.4
6,2001806273,486886,121003,d,0.615,1.0,1091117222194,1.00,121003,486886,d,Forward charges,90.2,1.0,90.2
7,2001806304,302017,121003,b,0.500,0.5,1091117222360,0.71,121003,302017,d,Forward charges,90.2,1.0,33.0
8,2001806338,392150,121003,d,0.500,0.5,1091117222570,0.70,121003,392150,d,Forward charges,90.2,1.0,45.4
9,2001806408,532484,121003,d,2.265,2.5,1091117222931,2.50,121003,532484,d,Forward charges,224.6,2.5,224.6


In [ ]:
final['Difference Between Expected Charges and Billed Charges (Rs.)']=final['Expected_charge as per x(Rs)']-final['Billing Amount (Rs.)']
final.head()

In [ ]:
#Arrange the columns as per expected result:
final.rename(columns={'Zone':'Delivery Zone charged by Courier Company','Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)','Charged Weight':'Total weight as per Courier Company (KG)'},inplace=True)
report=final.iloc[:,[0,6,4,5,7,-3,3,10,-2,-4,-1]]
report

,Order ID,AWB Code,Total weight as per X (KG),weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected_charge as per x(Rs),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,b,d,89.6,135.0,-45.4
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,176.3,172.8,3.5
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,355.5,345.0,10.5


In [ ]:
report['Delivery Zone as per X']=report['Delivery Zone as per X'].str.upper()
report['Delivery Zone charged by Courier Company']=report['Delivery Zone charged by Courier Company'].str.upper()

<ipython-input-161-9654c86849ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['Delivery Zone as per X']=report['Delivery Zone as per X'].str.upper()
<ipython-input-161-9654c86849ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['Delivery Zone charged by Courier Company']=report['Delivery Zone charged by Courier Company'].str.upper()


In [ ]:
report.head(2)

,Order ID,AWB Code,Total weight as per X (KG),weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected_charge as per x(Rs),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.5,2.92,3.0,B,B,33.0,174.5,-141.5
1,2001806226,1091117222065,0.48,0.5,0.68,1.0,D,D,45.4,90.2,-44.8


In [ ]:
report.to_excel('Expected Result.xlsx',sheet_name='calculation',index=False)

In [ ]:
df=pd.read_excel('/content/Expected Result.xlsx')

In [ ]:
df

,Order ID,AWB Code,Total weight as per X (KG),weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected_charge as per x(Rs),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.),charges_check
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,B,B,33.0,174.5,-141.5,over charged
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,D,D,45.4,90.2,-44.8,over charged
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,D,D,45.4,90.2,-44.8,over charged
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,135.0,135.0,0.0,correctly Charged
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,33.0,61.3,-28.3,over charged
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,B,D,33.0,45.4,-12.4,over charged
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,B,D,89.6,135.0,-45.4,over charged
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,D,D,176.3,172.8,3.5,under Charged
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,D,D,355.5,345.0,10.5,under Charged


In [ ]:
#check the charges for over rated and under rated:
def cout(x,y):
  if(x==y):
    return "correctly Charged"
  elif(x>y):
    return "under Charged"
  else:
    return "over charged"


df['charges_check']=final.apply(lambda x: cout(x['Expected_charge as per x(Rs)'],x['Charges Billed by Courier Company (Rs.)']),axis=1)


In [ ]:
df.groupby(by='charges_check')['Charges Billed by Courier Company (Rs.)'].sum()


charges_check
correctly Charged    1826.9
over charged         8469.8
under Charged        3351.5
Name: Charges Billed by Courier Company (Rs.), dtype: float64

In [ ]:
#create dataframe count and sum the amount from difference calculated column:
summary=pd.DataFrame({'count' : df.groupby(by='charges_check').size(),'Amount':df.groupby(by='charges_check')['Difference Between Expected Charges and Billed Charges (Rs.)'].sum(),'Total Amount charged by invoice':df.groupby(by='charges_check')['Charges Billed by Courier Company (Rs.)'].sum()}).reset_index()
summary


,charges_check,count,Amount,Total Amount charged by courier
0,correctly Charged,22,0.0,1826.9
1,over charged,79,-4426.6,8469.8
2,under Charged,23,630.6,3351.5


save the final dataframe in excel format:

In [ ]:

with pd.ExcelWriter('Expected Result.xlsx') as writer:
  
  report.to_excel(writer, sheet_name='calculations',index =False)
  summary.to_excel(writer, sheet_name='summary',index =False)